<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-Font_Fix

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the Font Fix conservative-to-primitive routine available in GRHayL.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#introduction): **Introduction**
1. [Step 2](#Noble_Routines): **`Noble Routines`**
    1. [Step 2.a](#variables_needed_by_Noble): *Setting up the variables needed by `Noble`*
        1. [Step 2.a.i](#Noble_vars): Conversion to Noble variables
        1. [Step 2.a.ii](#Noble_B): Computing $B^{i}_{\rm Noble}$
        1. [Step 2.a.iii](#Noble_prims): Writing $\boldsymbol{P}_{\rm Noble}$ in terms of $\boldsymbol{P}_{\rm GRHayL}$
    1. [Step 2.b](#equatorial_symmetry): *Applying equatorial symmetry*
    1. [Step 2.c](#variable_setup): *Setting up the variables needed by `HARM`*
        1. [Step 2.c.ii](#variable_setup__prims): Primitives
        1. [Step 2.c.iii](#variable_setup__conservs): Conservatives
        1. [Step 2.c.iv](#variable_setup__lapse_and_psi): Lapse function and conformal factor
        1. [Step 2.c.v](#variable_setup__phys_metric): Physical spatial metric
        1. [Step 2.c.vi](#variable_setup__betadown_and_beta2): $\beta_{i}$ and $\beta^{2} \equiv \beta_{i}\beta^{i}$
        1. [Step 2.c.vii](#variable_setup__adm_4metric): The ADM 4-metric, $g_{\mu\nu}$, and its inverse, $g^{\mu\nu}$
        1. [Step 2.c.viii](#variable_setup__temp_conservs): Temporary storage for current values of the conservative variables
    1. [Step 2.d](#conserv_to_prim__driver): *Determining the primitives variables from the conservatives variables*
    1. [Step 2.e](#enforce_limits_on_primitives_and_recompute_conservs): *Enforcing physical limits on primitives and recomputing the conservatives variables*
    1. [Step 2.f](#updating_conservs_and_prims_gfs): *Updating conservative and primitive gridfunctions*
    1. [Step 2.g](#diagnostics_and_debugging_tools): *Diagnostics and debugging tools*
1. [Step 3](#harm_primitives_lowlevel): **`harm_primitives_lowlevel.C`**
1. [Step 7](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

The `font_fix` conservative-to-primitive (C2P) function for the Hybrid EOS that can serve as a failsafe in the event of the failure of the other C2P methods. This function isn't included in the `Hybrid_Multi_Method`. To use it, we simply call `font_fix` directly, and it is usually triggered only if the return value from the main C2P method(s) is(are) non-zero. While `font_fix` has the same argument list as the other C2P functions, it does not require initial primitive guesses. The only necessary primitive inputs are $B^i$.

For more information on the arguments for conservative-to-primitive routines, see the [tutorial](../Tutorial-Calling_Con2Prim_Routines.ipynb) on Con2Prim routines.

<a id='font_fix'></a>

# Step 2: The `font_fix` Function \[Back to [top](#toc)\]
$$\label{font_fix}$$

The `font_fix` function implements the procedure suggested by [Font *et al.* (1998)](https://arxiv.org/abs/gr-qc/9811015). The algorithm can be summarized as the requirement that

$$
P=P_{\rm cold} = \kappa \rho^{\Gamma_{\rm cold}}_{b}\ ,
$$

and then recomputing the velocities $u_{i}$. We will describe this procedure in detail in [Step 4](#font_fix_gamma_law__c) below.

## <a id='compute_v'></a>

## Step 2.d: Compute v \[Back to [top](#toc)\]
$$\label{compute_v}$$

Now we compute

$$
\tilde{u}^{i} = \gamma^{ij}u_{i} \implies
\boxed{
\left\{
\begin{align}
\tilde{u}^{x} &= \gamma^{xx}u_{x} + \gamma^{xy}u_{y} + \gamma^{xz}u_{z}\\
\tilde{u}^{y} &= \gamma^{yx}u_{x} + \gamma^{yy}u_{y} + \gamma^{yz}u_{z}\\
\tilde{u}^{z} &= \gamma^{zx}u_{x} + \gamma^{zy}u_{y} + \gamma^{zz}u_{z}
\end{align}
\right.
}
$$

and use the `limit_utilde_and_compute_v` function to speed limit $\tilde{u}^i$ and compute v.

<a id='primitives'></a>

## Step 3.f: Setting the primitives \[Back to [top](#toc)\]
$$\label{primitives}$$

Finally, we update the remaining primitives,

$$
\boxed{
\begin{align}
\rho_{b} &= \frac{\tilde{\rho}}{\alpha \psi^6 u^0} \\
P &= P_{\rm cold} \\
\epsilon &= \frac{P}{\rho_b(\Gamma-1)}
\end{align}
}
$$

<a id='font_fix_hybrid_eos'></a>

# Step 4: `font_fix_hybrid_EOS.C` \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos}$$

The [Font *et al.*](https://arxiv.org/pdf/gr-qc/9811015.pdf) algorithm (henceforth Font Fix algorithm) is implemented in the `font_fix_hybrid_EOS` function and can be summarized as follows. Font fixes occur at the atmospheric region, so we start by assuming that $P$ is given only by its cold part, i.e.

$$
P = P_{\rm cold} = K_{\rm atm} \rho_{b}^{\Gamma_{\rm atm}}\ ,
$$

where $K_{\rm atm}$ and $\Gamma_{\rm atm}$ are the constants used in the atmosphere. Then, the specific internal energy is given by

$$
\begin{align}
\epsilon &= \epsilon_{\rm cold}\\
         &= \int d\rho \frac{P_{\rm cold}}{\rho^{2}}\\
         &= K_{\rm atm}\int d\rho \rho^{\Gamma_{\rm atm}-2}\\
         &= \frac{K_{\rm atm}\rho^{\Gamma_{\rm atm}-1}}{\Gamma_{\rm atm}-1}\ .
\end{align}
$$

Having computed $P$ and $\epsilon$, we can compute the enthalpy, $h$, giving

$$
\begin{align}
h &= 1 + \epsilon + \frac{P}{\rho}\\
  &= 1 + \frac{K_{\rm atm}\rho^{\Gamma_{\rm atm}-1}}{\Gamma_{\rm atm}-1} + K_{\rm atm}\rho^{\Gamma_{\rm atm} - 1}\\
  &= 1 + \left(\frac{1}{\Gamma_{\rm atm}-1}+1\right)K_{\rm atm}\rho^{\Gamma_{\rm atm} - 1}\\
\implies &\boxed{ h = 1 + \left(\frac{\Gamma_{\rm atm}}{\Gamma_{\rm atm}-1}\right)K_{\rm atm} \rho^{\Gamma_{\rm atm}-1} }\ .
\end{align}
$$

We then run an iterative process that updates $\rho$ in a consistent way, based on the value of $h$. We now describe this process and its implementation.

<a id='font_fix_hybrid_eos__basic_quantities'></a>

## Step 4.a: Computing the basic quantities needed by the algorithm \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__basic_quantities}$$

We start by translating GRHayL's $B$ to that expected by the Font Fix algorithm
$$\bar{B}^{i} = \frac{B^i}{\sqrt{4\pi}}$$
and computing several quantities derived from $\bar{B}^{i}$.

<a id='font_fix_hybrid_eos__sdots'></a>

## Step 4.b: Basic check: looking at $\tilde{S}^{2}$ \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__sdots}$$

We start by looking at the dot product $\tilde{S}^{2}$. Recall that

$$
\tilde{S}_{i} = \left(\tilde{\rho}h + \alpha\sqrt{\gamma}\, u^{0}b^{2}\right)u_{i}-\alpha\sqrt{\gamma}\, b^{0}b_{i}\ .
$$

If $\tilde{S}^{2} = 0$, then we must be in a region where $u_{i} = 0 = b_{i}$. In this case, we return

$$
\begin{align}
\rho_{b} &= \psi^{-6}\tilde{\rho}\ ,\\
u_{i} &= 0\ ,
\end{align}
$$

and terminate the function call.

<a id='font_fix_hybrid_eos__initial_guesses'></a>

## Step 4.c: Initial guesses for $W$, $S_{{\rm fluid}}^{2}$, and $\rho$ \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__initial_guesses}$$

If $\tilde{S}^{2} \neq 0$, then we move on to the iterative procedure previously mentioned. We start by setting the initial data based on eqs. (A52), (A53), and (A59) found in [Appendix A of Zachariah *et al.* (2012)](https://arxiv.org/pdf/1112.0568.pdf):

$$
\boxed{
\begin{align}
W_{0} &= \psi^{-6}\sqrt{\left(\hat{\bar{B}}\cdot\tilde{S}\right)^{2} + \tilde{\rho}^{2}}\\
S_{{\rm fluid},0}^{2} &=\frac{W_{0}^{2}\left(\tilde{S}\cdot\tilde{S}\right)+\left(\bar{B}\cdot\tilde{S}\right)^{2}\left(\bar{B}^{2} + 2W_{0}\right)}{\left(W_{0} + \bar{B}^{2}\right)^{2}}\\
\rho_{0} &= \frac{\psi^{-6}\tilde{\rho}}{\sqrt{1+\frac{S_{{\rm fluid},0}^{2}}{\tilde{\rho}^{2}}}}
\end{align}
}\ .
$$

<a id='font_fix_hybrid_eos__main_loop'></a>

## Step 4.d: The main loop \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__main_loop}$$

We now perform the following iterative process, which is again described in [Appendix A of Zachariah *et al.* (2012)](https://arxiv.org/pdf/1112.0568.pdf). We refer the reader to eqs. (A60), (A61), and (A62).

1. Store the previously computed values of $W_{n}$, $S_{{\rm fluid},n}^{2}$, and $\rho_{n}$
2. Compute $h = 1 + \epsilon_{\rm cold} + \frac{P_{\rm cold}}{\rho_{n}}$
3. Set
$$
\boxed{\rho_{n+1} = \psi^{-6}\tilde{\rho}\left(1 + \frac{S_{{\rm fluid},n}^{2}}{\tilde{\rho} h_{n}}\right)^{-1/2}}
$$

4. For a given value of $n$, perform steps 1 (for $\rho$), 2 and 3 until $\left|\rho_{n+1}-\rho_{n}\right| < \rho_{n+1}\epsilon$, where $\epsilon$ is a user given tolerance
5. After convergence is obtained, update:
$$
\boxed{
\begin{align}
h_{n+1} &= 1 + \epsilon_{\rm cold} + P_{\rm cold}/\rho_{n+1}\\
W_{n+1} &= \psi^{-6}\sqrt{\tilde{S}^{2}_{{\rm fluid},n} + \tilde{\rho}^{2} h_{n+1}^{2}}\\
S_{{\rm fluid},n+1}^{2} &= \frac{W^{2}_{n+1}\left(\tilde{S}\cdot\tilde{S}\right) + \left(\bar{B}\cdot\tilde{S}\right)^{2}\left(\bar{B}^{2} + 2W_{n+1}\right)}{\left(W_{n+1} + \bar{B}^{2}\right)^{2}}
\end{align}
}\ .
$$
6. Repeat steps 1 through 5 until $\left|W_{n+1}-W_{n}\right| < W_{n+1}\epsilon$ *and* $\left|S^{2}_{{\rm fluid},n+1}-S^{2}_{{\rm fluid},n}\right| < S^{2}_{{\rm fluid},n+1}\epsilon$ *or* we reach the maximum number of iterations
7. If font fix fails, increase the tolerance and try again.

This is done using the function `font_fix__rhob_loop()`, which is documented in the [inlined functions tutorial notebook](Tutorial-IllinoisGRMHD__inlined_functions.ipynb).

<a id='font_fix_hybrid_eos__outputs'></a>

## Step 4.e: Output $\rho_{b}$ and $u_{i}$ \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__outputs}$$

Finally, we return $\rho_{b}$ and $u_{i}$. In the relations below, $N$ indicates the last computed value of $\rho$ obtained by our iterative process. The quantities evaluated here are

$$
\boxed{
\begin{align}
\rho_{b}   &= \rho_{N}\\
\gamma_{v} &= \frac{\psi^{-6}\rho_{\star}}{\rho_{b}}\\
f_{1}      &= \frac{\psi^{6}\left(\bar{B}\cdot\tilde{S}\right)}{\gamma_{v}\rho_{\star} h}\\
f_{2}      &= \left(\rho_{\star}h + \psi^{6}\frac{\bar{B}^{2}}{\gamma_{v}}\right)^{-1}\\
u_{i}      &= f_{2}\left(\tilde{S}_{i} + f_{1}\bar{B}_{i}\right)
\end{align}
}\ .
$$
